In [138]:
import pandas as pd

In [139]:
df_wetter = pd.read_csv("Daten/Wetter_2024/clean_wetter_komplett.csv", header = 0)

In [140]:
df_wetter.set_index("Unnamed: 0", inplace = True)

In [141]:
df_messungen = pd.read_csv("Daten/messungen_clean_2024.csv", header = 0)

In [142]:
df_messungen.set_index("DATUM", inplace = True)

In [143]:
df_komplett = pd.merge(df_wetter, df_messungen, how = "left", left_index=True, right_index=True)

In [144]:
df_komplett.index.rename("zeit", inplace=True)
df_komplett.index = pd.to_datetime(df_komplett.index)

In [145]:
zeit_max = df_komplett[pd.notna(df_komplett["ecoli"])].index.max()
zeit_min = df_komplett[pd.notna(df_komplett["ecoli"])].index.min()

In [146]:
df_komplett = df_komplett[df_komplett.index > zeit_min- pd.Timedelta("7d")]
df_komplett = df_komplett[df_komplett.index < zeit_max + pd.Timedelta("7d")]

In [147]:
df_wendlingen= pd.read_csv("Daten/Wetter_2024/wendlingen_messwerte.csv", delimiter = ";", parse_dates = True, index_col = 0)

In [148]:
df_wendlingen = df_wendlingen.pivot(index = "Datum", columns = "Parameter", values = "Tagesmittelwert")

In [149]:
df_wendlingen.index = pd.to_datetime(df_wendlingen.index, format = "%d.%m.%Y")

In [150]:
df_wendlingen = df_wendlingen.resample("1h").first().ffill()

In [151]:
for col in df_wendlingen:
    df_wendlingen[col] = df_wendlingen[col].str.replace(",", ".")
    try:
        df_wendlingen[col] = pd.to_numeric(df_wendlingen[col])
    except:
        pass

In [152]:
df_komplett = df_komplett.merge(df_wendlingen, left_index=True, right_index=True)

In [153]:
df_komplett.reset_index(inplace = True)

In [154]:
df_komplett.columns = [col.strip() for col in df_komplett.columns]

In [155]:
df_komplett.to_csv("Daten/masterdata_2024.csv")